# Предварительная обработка

## Обработка пропусков


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro


: 

In [ ]:
# Загрузка данных (пример)
df = pd.read_csv('data.csv')

# 1. Заполнение средним/медианой/модой
# Для числовых столбцов
df_filled_mean = df.fillna(df.mean())  # средним
df_filled_median = df.fillna(df.median())  # медианой

In [ ]:
# Для категориальных столбцов
df_filled_mode = df.fillna(df.mode().iloc[0])  # модой

# 2. Удаление строк с пропусками

In [ ]:
# 2. Удаление строк с пропусками
df_dropped = df.dropna()  # удалить все строки с пропусками
df_dropped_rows = df.dropna(axis=0)  # то же самое
df_dropped_columns = df.dropna(axis=1)  # удалить столбцы с пропусками


# 3. Модели машинного обучения для заполнения пропусков

In [ ]:
# 3. Модели машинного обучения для заполнения пропусков
# Линейная регрессия для конкретного столбца
# Пример: заполним пропуски в столбце 'target' на основе других признаков
df_reg = df.copy()
train = df_reg.dropna(subset=['target'])
X_train = train.drop('target', axis=1)
y_train = train['target']
model = LinearRegression()
model.fit(X_train.fillna(X_train.mean()), y_train)


# 4. Алгоритм KNN (K-ближайших соседей) - NONA

In [ ]:
# 4. Алгоритм KNN (K-ближайших соседей) - NONA
imputer = KNNImputer(n_neighbors=5)
df_knn = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)


# SimpleImputer (аналогично заполнению средним/медианой/модой)

In [ ]:
# SimpleImputer (аналогично заполнению средним/медианой/модой)
imputer_mean = SimpleImputer(strategy='mean')
df_simple = pd.DataFrame(imputer_mean.fit_transform(df), columns=df.columns)

## Проверка на нормальное распределение и выбросы


# 1. Ящик с усами (Boxplot) для обнаружения выбросов

In [ ]:
# 1. Ящик с усами (Boxplot) для обнаружения выбросов
plt.figure(figsize=(12, 6))
df_numeric = df.select_dtypes(include=[np.number])  # только числовые столбцы
sns.boxplot(data=df_numeric)
plt.title('Ящики с усами для обнаружения выбросов')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# Вычисление границ выбросов по IQR (межквартильному размаху)

In [ ]:
for column in df_numeric.columns:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    print(f"{column}: найдено {len(outliers)} выбросов")

# 2. Фильтр Хемпеля для устранения выбросов (замена на граничные значения)

In [ ]:
df_hempel = df.copy()
for column in df_numeric.columns:
    Q1 = df_hempel[column].quantile(0.25)
    Q3 = df_hempel[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Замена выбросов на граничные значения
    df_hempel.loc[df_hempel[column] < lower_bound, column] = lower_bound
    df_hempel.loc[df_hempel[column] > upper_bound, column] = upper_bound


# 3. Гистограммы частот для проверки распределения

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, column in enumerate(df_numeric.columns):
    if i < len(axes):
        df[column].hist(ax=axes[i], bins=30, edgecolor='black', alpha=0.7)
        axes[i].set_title(f'Гистограмма: {column}')
        axes[i].axvline(df[column].mean(), color='red', linestyle='--', label='Среднее')
        axes[i].axvline(df[column].median(), color='green', linestyle='--', label='Медиана')
        axes[i].legend()

plt.tight_layout()
plt.show()


# 4. Проверка на нормальное распределение - тест Шапиро-Уилка

In [1]:
# Shapiro-Wilk работает только для 3-5000 наблюдений
# p-value > 0.05 = нормальное распределение
# Для n>5000 используй scipy.stats.kstest (Колмогоров-Смирнов)
for column in df_numeric.columns:
    data = df[column].dropna()
    n = len(data)  # вот это n!
    
    # Стандартизируем данные
    data_standardized = (data - data.mean()) / data.std()
    
    # K-S тест (сравнение с нормальным распределением)
    stat, p_value = kstest(data_standardized, 'norm')
    
    print(f"{column} (n={n}):")
    print(f"  Статистика: {stat:.4f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Нормальное: {'ДА' if p_value > 0.05 else 'НЕТ'}")
    print()

NameError: name 'df_numeric' is not defined

In [ ]:
# шапиро
print("Тест Шапиро-Уилка:")

shapiro_results = []
for column in df_numeric.columns:
    data = df[column].dropna()
    if 3 <= len(data) <= 5000:  # корректный диапазон для теста
        stat, p_value = shapiro(data)
        shapiro_results.append({
            'Столбец': column,
            'Статистика': f"{stat:.4f}",
            'p-value': f"{p_value:.4f}",
            'Нормальное': 'ДА' if p_value > 0.05 else 'НЕТ'
        })

shapiro_df = pd.DataFrame(shapiro_results)
print(shapiro_df)



# Q-Q plots для визуальной проверки


In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
axes = axes.flatten()

for i, column in enumerate(df_numeric.columns):
    if i < len(axes):
        stats.probplot(df[column].dropna(), dist="norm", plot=axes[i])
        axes[i].set_title(f'Q-Q plot: {column}')

plt.tight_layout()
plt.show()

# MINI!

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import shapiro

# Данные
df = pd.read_csv('data.csv')
df_numeric = df.select_dtypes(include=[np.number])

# 1. Выбросы через IQR
for col in df_numeric.columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)]
    print(f"{col}: {len(outliers)} выбросов")

# 2. Фильтр Хемпеля
df_clean = df.copy()
for col in df_numeric.columns:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    df_clean[col] = df_clean[col].clip(lower, upper)

# 3. Тест Шапиро-Уилка
print("\nТест Шапиро-Уилка:")
for col in df_numeric.columns:
    data = df[col].dropna()
    if len(data) >= 3:
        stat, p = shapiro(data)
        print(f"{col}: p={p:.4f} {'✓' if p > 0.05 else '✗'}")

## Трансформация данных

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# 1. OneHotEncoder для категориальных признаков
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

# Создаем энкодер
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_data = encoder.fit_transform(df[categorical_cols])

# Создаем DataFrame с закодированными признаками
encoded_df = pd.DataFrame(encoded_data, 
                         columns=encoder.get_feature_names_out(categorical_cols))

# Объединяем с исходными числовыми признаками
numerical_cols = df.select_dtypes(include=[np.number]).columns
df_encoded = pd.concat([df[numerical_cols], encoded_df], axis=1)


In [ ]:
# 2. GroupBy - группировка данных
# Простая группировка с агрегацией
grouped = df.groupby('category_column').agg({
    'numeric_column1': 'mean',
    'numeric_column2': ['sum', 'count'],
    'numeric_column3': 'std'
})

# Группировка с одной агрегацией
grouped_simple = df.groupby('category_column')['numeric_column'].mean().reset_index()
print("\nПростая группировка:")
print(grouped_simple.head())



In [ ]:
# 3. Pivot Table - сводная таблица
pivot_table = pd.pivot_table(
    df,
    values='value_column',
    index='row_category',
    columns='column_category',
    aggfunc='mean',
    fill_value=0
)


In [ ]:
# Сводная таблица с несколькими агрегациями
pivot_multi = pd.pivot_table(
    df,
    values=['value1', 'value2'],
    index='row_category',
    columns='column_category',
    aggfunc={'value1': 'mean', 'value2': 'sum'},
    fill_value=0
)


# Mini

In [ ]:
# 1. OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
encoded = encoder.fit_transform(df[['category_col']])
df_encoded = pd.concat([df, pd.DataFrame(encoded, columns=encoder.get_feature_names_out())], axis=1)

# 2. GroupBy
grouped = df.groupby('cat_col')['num_col'].mean().reset_index()

# 3. Pivot Table
pivot = pd.pivot_table(df, values='value', index='row', columns='col', aggfunc='mean')

In [ ]:
# OneHot
from sklearn.preprocessing import OneHotEncoder
df = pd.concat([df, pd.DataFrame(OneHotEncoder(sparse_output=False).fit_transform(df[['cat']]))], axis=1)

# GroupBy
grouped = df.groupby('cat')['num'].agg(['mean', 'sum'])

# Pivot
pivot = pd.pivot_table(df, values='val', index='row', columns='col', aggfunc='mean')

## Понижение размерности

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler


In [ ]:
# Масштабирование ВСЕГДА перед PCA/UMAP
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_numeric)

In [ ]:
# 1. PCA - быстро, линейное
pca = PCA(n_components=2)  # 2 компоненты для визуализации
X_pca = pca.fit_transform(X_scaled)
print(f"PCA: объяснено {pca.explained_variance_ratio_.sum():.1%} дисперсии")

In [ ]:
# 2. t-SNE - для визуализации, медленно
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)
print("t-SNE готов")

In [ ]:
# 3. UMAP - быстрый t-SNE
reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_scaled)
print("UMAP готов")


In [ ]:
print("- PCA: если >100 признаков, для ускорения моделей")
print("- t-SNE: ТОЛЬКО для визуализации (2D/3D)")
print("- UMAP: лучший выбор для визуализации и кластеризации")

# Mini

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.select_dtypes(include=[np.number]).fillna(0))

pca = PCA(n_components=0.95)  # оставить 95% дисперсии
X_pca = pca.fit_transform(X_scaled)
print(f"Сжали {X_scaled.shape[1]} признаков до {X_pca.shape[1]}")

In [ ]:
# Всегда проверяй сколько дисперсии объясняют компоненты
pca = PCA().fit(X_scaled)  # все компоненты
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# Найди сколько компонент нужно для 95% дисперсии
n_components = np.argmax(cumulative_variance >= 0.95) + 1
print(f"Нужно {n_components} компонент для 95% дисперсии")